# Access Relational DB
- model: relational algebra
- standard SQL
- editor: PostgreSQL, MySQL/MariaDB, SQL Server, Oracle DB

In [2]:
import psycopg2 as pg

In [1]:
host = "10.31.70.139"
port = 5432
dbname = "dbmovie"
user = "movie"
password = "password"

In [3]:
# postgresql://someone@example.com/somedb
url = f"postgresql://{host}:{port}/{dbname}"

In [4]:
conn = pg.connect(url, user=user, password=password)

In [5]:
conn

<connection object at 0x0000022CD1117BF0; dsn: 'user=movie password=xxx dbname=dbmovie host=10.31.70.139 port=5432', closed: 0>

In [6]:
query_all_movies = "select * from movies"

In [7]:
cur = conn.cursor()

In [9]:
cur.execute(query_all_movies)

In [10]:
# cursor is iterable
movies = list(cur)
movies[:3]

[(5257,
  'Excuse Me',
  1915,
  50,
  'Henry Mallory, U.S.A., receives orders to join his regiment which is to embark for the Philippines. The Overland Limited is the only train that will enable him to reach the coast in time to escape a court-martial. Having a little time to spare he persuades Marjorie to elope with him and reserves two berths. They reach the train in time, but haven\'t time to get married. "Little" Jimmy Wellington, who stands six feet two and weighs 350 in his stocking feet, guzzles too much, so his wife takes the Overland for Reno, telling him she goes to Paris for the year. He gets sore and takes the same train for the same place and for the same purpose. The Reverend Doctor Temple, tiring of the Monotony of Ypsilanti elopes and takes the Overland to the coast. There is another girl aboard, a former sweetheart of Mallory\'s. With this cargo in charge of a worthy porter, they start. The happenings en route are screamingly funny. Marjorie, not married to Mallory, h

In [11]:
cur.close()
conn.close()